In [1]:
import pandas as pd
import numpy as np
import requests
from requests import get
import json

import env

# Twitter API

# Notes:

Hootsuite: https://www.youtube.com/watch?v=Ztk6PXD1EjU

In [5]:
url = "https://api.twitter.com/2/users/61261275/tweets?user.fields=created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld&max_results=100&start_time=2020-09-30T00:00:00Z&end_time=2020-10-15T00:00:00Z&expansions=&tweet.fields=created_at,public_metrics,source,text"

payload = {}
headers = {"Authorization": "Bearer {}".format(env.bearer_token)}

response = requests.request("GET", url, headers=headers, data = payload)

print(response.text.encode('utf8'))

b'{"data":[{"text":"Have you registered for the Great #California #Shakeout #Earthquake Preparedness Drill this Thursday, October 15? Plan to be safe and learn about how to prepare and secure your space for earthquakes. More info here: https://t.co/Q0iVvUIDbG  @ECA @ReadyLA @NotifyLA https://t.co/3rzXvNLWW2","created_at":"2020-10-14T20:30:07.000Z","source":"Hootsuite Inc.","public_metrics":{"retweet_count":1,"reply_count":3,"like_count":1,"quote_count":0},"id":"1316476400401092608"},{"text":"#LADOT has introduced economic relief measures to assist #Angelenos in need as parking enforcement resumes this Thursday, October 15. For more information, visit: https://t.co/GG2HFz3xyT https://t.co/jEk7nl7CVU","created_at":"2020-10-14T19:35:15.000Z","source":"Twitter Web App","public_metrics":{"retweet_count":14,"reply_count":22,"like_count":8,"quote_count":12},"id":"1316462593825894400"},{"text":"@YourDreamFit @YourDreamFit Thank you for reaching out. Please try this link https://t.co/GG2HFzl8Xt

In [8]:
def get_params():
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    return {"tweet.fields": "created_at"}


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", url, headers=headers, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


def main():
    bearer_token = auth()
    url = create_url()
    headers = create_headers(bearer_token)
    params = get_params()
    json_response = connect_to_endpoint(url, headers, params)
    print(json.dumps(json_response, indent=4, sort_keys=True))

In [9]:
main()

200
{
    "data": [
        {
            "created_at": "2021-01-06T18:40:40.000Z",
            "id": "1346889436626259968",
            "text": "Learn how to use the user Tweet timeline and user mention timeline endpoints in the Twitter API v2 to explore Tweets and mentions \ud83d\udc47\n\nhttps://t.co/8ef0gXruSD"
        },
        {
            "created_at": "2021-01-05T22:45:35.000Z",
            "id": "1346588685555306497",
            "text": "\ud83d\udd11\ud83d\udd11 On Tuesday, January 12th, we\u2019re removing the ability to view existing consumer API keys from the developer portal. Be sure to save your API keys in a secure place before Tuesday to ensure your access to the #TwitterAPI is not disrupted. Learn more https://t.co/UztnVZrVwa"
        },
        {
            "created_at": "2020-12-23T15:04:28.000Z",
            "id": "1341761599976181763",
            "text": "\ud83d\udcc8 From Tweets to Google Sheets: @JessicaGarson explains how to use Python to seamlessly turn a 